# Install

In [1]:

# @title Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

ValueError: mount failed

In [ ]:

# @title Clone
!git clone https://github.com/yaranbarzi/Applio --branch 3.2.9 --single-branch
%cd /content/Applio

In [ ]:

# @title Install

!sudo apt update
!sudo apt install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3
import sys
sys.path.append('/usr/local/lib/python3.10/dist-packages')

rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)

!pip install uv
from IPython.display import clear_output
clear_output()
print("Installing requirements...")
!uv pip install torch==2.7.0 torchvision torchaudio==2.7.0 --upgrade --index-url https://download.pytorch.org/whl/cu128 -q
!uv pip install -r requirements.txt -q
clear_output()
print("Finished installing requirements! ")

In [ ]:

# @title Download models
!python core.py "prerequisites" --models "True" --exe "True"  --pretraineds_hifigan "True"

# Infer

In [ ]:

# @title Download model
# @markdown Hugging Face or Google Drive
model_link = "https://drive.google.com/file/d/15t0QOye_o5ha6QqTpiSqiex1CRHlLFW6/view?usp=sharing"  # @param {type:"string"}

!python core.py download --model_link "{model_link}"

In [ ]:


# @title 🎤 Run Inference (Simplified)

#@markdown ---
#@markdown ### ۱. تنظیمات اصلی
#@markdown نام مدلی که در سلول قبل زیپ کردید (بدون پسوند .zip)
model_name = ""  # @param {type:"string"}
#@markdown مسیر کامل فایل صوتی که می‌خواهید صدایش را تغییر دهید.
input_path = ""  # @param {type:"string"}
#@markdown فرمت فایل خروجی نهایی
export_format = "WAV"  # @param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A']

#@markdown ---
#@markdown ### ۲. تنظیمات تبدیل صدا
#@markdown تغییر زیر و بمی صدا (Pitch). اعداد مثبت صدا را زیرتر و اعداد منفی آن را بم‌تر می‌کنند.
f0_up_key = 0  # @param {type:"slider", min:-24, max:24, step:1}
#@markdown میزان اعمال ویژگی‌های صدای مدل. مقادیر کمتر می‌تواند به کاهش آرتیفکت کمک کند.
index_rate = 0.5  # @param {type:"slider", min:0.0, max:1.0, step:0.05}
#@markdown متد استخراج Pitch. Rmvpe معمولاً بهترین کیفیت را دارد.
f0_method = "rmvpe"  # @param ["rmvpe", "crepe", "crepe-tiny"]

#@markdown ---
#@markdown ### ۳. تنظیمات پیشرفته
#@markdown اگر فایل صوتی ورودی بسیار طولانی است، این گزینه را فعال کنید.
split_audio = False  # @param{type:"boolean"}

# ------------------- Internal Settings (Hardcoded for simplicity) -------------------
# این مقادیر روی حالت بهینه تنظیم شده و از UI حذف شده‌اند.
rms_mix_rate = 0.8
protect = 0.5
hop_length = 128
clean_audio = False
clean_strength = 0.7
f0_autotune = False
formant_shift = False
formant_qfrency = 1.0
formant_timbre = 1.0
embedder_model = "contentvec"
embedder_model_custom = ""
output_path = "/content/output.wav"
# Disable all post-processing effects
post_process = False
# ------------------------------------------------------------------------------------

import os
from IPython.display import Audio, display, clear_output

# Find model and index files automatically
model_folder = f"/content/Applio/logs/{model_name}"

if not os.path.exists(model_folder):
    raise FileNotFoundError(f"خطا: پوشه مدل '{model_folder}' پیدا نشد. لطفاً از درستی نام مدل مطمئن شوید.")

try:
    files_in_folder = os.listdir(model_folder)
    pth_file_name = next((f for f in files_in_folder if f.endswith(".pth")), None)
    index_file_name = next((f for f in files_in_folder if f.endswith(".index")), None)

    if pth_file_name is None or index_file_name is None:
        raise FileNotFoundError("فایل مدل (.pth) یا فایل ایندکس (.index) در پوشه پیدا نشد.")

    pth_path = os.path.join(model_folder, pth_file_name)
    index_path = os.path.join(model_folder, index_file_name)

    print("✅ فایل‌های مدل با موفقیت پیدا شدند:")
    print(f"   > Model: {pth_file_name}")
    print(f"   > Index: {index_file_name}")
    print("\nدر حال اجرای تبدیل صدا...")

    # Run the inference command
    !python core.py infer \
        --pitch "{f0_up_key}" \
        --volume_envelope "{rms_mix_rate}" \
        --index_rate "{index_rate}" \
        --hop_length "{hop_length}" \
        --protect "{protect}" \
        --f0_autotune "{f0_autotune}" \
        --f0_method "{f0_method}" \
        --input_path "{input_path}" \
        --output_path "{output_path}" \
        --pth_path "{pth_path}" \
        --index_path "{index_path}" \
        --split_audio "{split_audio}" \
        --clean_audio "{clean_audio}" \
        --clean_strength "{clean_strength}" \
        --export_format "{export_format}" \
        --embedder_model "{embedder_model}" \
        --embedder_model_custom "{embedder_model_custom}" \
        --formant_shifting "{formant_shift}" \
        --formant_qfrency "{formant_qfrency}" \
        --formant_timbre "{formant_timbre}" \
        --post_process "{post_process}"

    # Display the output audio
    final_output_path = f"/content/output.{export_format.lower()}"
    if os.path.exists(final_output_path):
        clear_output()
        print(f"✅ تبدیل صدا با موفقیت انجام شد! فایل خروجی:")
        display(Audio(final_output_path, autoplay=True))
    else:
        print("❌ خطا: فایل خروجی ایجاد نشد. لطفاً لاگ‌های بالا را برای یافتن خطا بررسی کنید.")

except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"یک خطای غیرمنتظره رخ داد: {e}")